In [50]:
%matplotlib inline 
import glob
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.classes = os.listdir(root_dir)
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        #print(self.class_to_idx)
        self.transform = transform
        self.samples = self._make_dataset()
        #print(self.samples)
        
    def _make_dataset(self):
        samples = []
        for class_name in self.classes:
            class_dir = os.path.join(self.root_dir, class_name)
            if not os.path.isdir(class_dir):
                continue
            class_idx = self.class_to_idx[class_name]
            image_files = os.listdir(class_dir)
            for image_file in image_files:
                image_path = os.path.join(class_dir, image_file)
                samples.append((image_path, class_idx))
        return samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        image_path, class_idx = self.samples[index]
        image = Image.open(image_path)
        image_np = np.array(image)
        image = torch.from_numpy(image_np)
        print(image_path)
        #image = torchvision.io.read_image(image_path)
        #print(image)
        # Perform any necessary preprocessing on the image
        # For example, you can apply transformations using torchvision.transforms

        # Convert PIL image to torch tensor
        #image = Image.open(image_path)
        #image_np = np.array(image)
        #image = torch.from_numpy(image_np)
        #image = torch.tensor(image)

        #if self.transform is not None:
        #    image = self.transform(image)
            
        return image, class_idx
    

In [51]:
transform = transforms.Compose(
  [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#trainset = droneplanesDataset(path='./drone_and_planes', train=True, transform=transform)
#trainloader = DataLoader(dataset=trainset, batch_size=1, shuffle=True, drop_last=False)
#testset = droneplanesDataset(path='./drone_and_planes', train=False, transform=transform)
#testloader = DataLoader(dataset=testset, batch_size=1, shuffle=False, drop_last=False)
batch_size = 32
trainset = CustomDataset(root_dir='/dataset',transform='PNG')
#sample = trainset[0]
#print(sample)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)

testset = CustomDataset(root_dir='/dataset_test',transform='PNG')
#sample = trainset[0]
#print(sample)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=True)

In [52]:
trainset

In [53]:
classes = ('ClassA', 'ClassB', 'ClassC', 'ClassD', 'ClassE')

In [54]:
import matplotlib.pyplot as plt
import numpy as np

In [55]:
def imshow(img):
  img = img / 2+ 0.5 # unnormalize 
  npimg = img.numpy()
  plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [56]:
dataiter = iter(trainloader)

In [57]:
images, class_idxs = next(dataiter)

/dataset\ClassA\image15.png
/dataset\ClassD\0020.jpg
/dataset\ClassA\image33.png
/dataset\ClassB\0__20170106_180852_0e30__-122.3311205965009_37.71336901658289.png
/dataset\ClassB\0__20161018_192144_0c24__-122.38801773887562_37.80999245445229.png
/dataset\ClassA\image23.png
/dataset\ClassB\0__20170106_180851_0e30__-122.32845530039546_37.72960461142898.png
/dataset\ClassC\image39.png
/dataset\ClassA\image6.png
/dataset\ClassB\0__20161212_180856_0e30__-122.33922969685835_37.766060539560605.png
/dataset\ClassC\image50.png
/dataset\ClassA\image36.png
/dataset\ClassB\0__20161018_192144_0c24__-122.40397212243812_37.81369751286148.png
/dataset\ClassE\pic_030.jpg
/dataset\ClassD\0024.jpg
/dataset\ClassA\image43.png
/dataset\ClassC\image48.png
/dataset\ClassB\0__20161006_004004_0c38__-122.34842215196089_37.758757021099356.png
/dataset\ClassC\image19.png
/dataset\ClassA\image35.png
/dataset\ClassE\pic_004.jpg
/dataset\ClassC\image12.png
/dataset\ClassA\image48.png
/dataset\ClassB\0__20161212_1808

RuntimeError: stack expects each tensor to be equal size, but got [32, 49, 4] at entry 0 and [450, 450, 3] at entry 1

In [58]:
imshow(torchvision.utils.make_grid(images))
print(' '.join('%s' % classes[labels[j]] for j in range(1)))

NameError: name 'images' is not defined

In [59]:
import torch.nn as nn 
import torch.nn.functional as F

In [60]:
class Net(nn.Module): 
    def __init__(self):
      super(Net, self).__init__()
      self.conv1 = nn.Conv2d(3, 6, 5) 
      self.pool = nn.MaxPool2d(2, 2) 
      self.conv2 = nn.Conv2d(6, 16, 5) 
      self.fc1 = nn.Linear(16* 5 * 5, 120) 
      self.fc2= nn.Linear(120, 84) 
      self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
      x = self.pool(F.relu(self.conv1(x))) 
      x = self.pool(F.relu(self.conv2(x))) 
      x = x.view(-1, 16 * 5 * 5) 
      print(x)
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x)) 
      x = self.fc3(x)
      return x

In [61]:
net = Net()
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [62]:
for epoch in range(2):
  running_loss = 0.0
  for i, data in enumerate(trainloader, 0): 
    inputs, labels = data
    optimizer.zero_grad() 
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    running_loss += loss.item()
    if i % 2000 == 1999:
      print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000)) 
      running_loss = 0.0

/dataset\ClassD\0024.jpg
/dataset\ClassD\0039.jpg
/dataset\ClassE\pic_015.jpg
/dataset\ClassE\pic_036.jpg
/dataset\ClassA\image9.png
/dataset\ClassD\0056.jpg
/dataset\ClassD\0028.jpg
/dataset\ClassA\image10.png
/dataset\ClassD\0063.jpg
/dataset\ClassB\0__20160905_193458_0c37__-122.33299988326662_37.74671934055966.png
/dataset\ClassA\image17.png
/dataset\ClassE\pic_054.jpg
/dataset\ClassA\image18.png
/dataset\ClassD\0037.jpg
/dataset\ClassA\image33.png
/dataset\ClassB\0__20170328_181231_0e0f__-122.34465394430563_37.72647807788947.png
/dataset\ClassA\image23.png
/dataset\ClassE\pic_013.jpg
/dataset\ClassC\image27.png
/dataset\ClassE\pic_020.jpg
/dataset\ClassA\image20.png
/dataset\ClassE\pic_048.jpg
/dataset\ClassA\image34.png
/dataset\ClassE\pic_044.jpg
/dataset\ClassC\image12.png
/dataset\ClassB\0__20170106_180852_0e30__-122.33754040846814_37.71102578191657.png
/dataset\ClassC\image36.png
/dataset\ClassB\0__20161202_180735_0e30__-122.33342895873801_37.74823597436849.png
/dataset\ClassE

RuntimeError: stack expects each tensor to be equal size, but got [1000, 1000, 3] at entry 0 and [183, 275, 3] at entry 1

In [63]:
print('Finished Training')

Finished Training


In [64]:
dataiter = iter(testloader) 
images, labels = next(dataiter)

/dataset_test\ClassE\pic_026.jpg
/dataset_test\ClassC\image49.png
/dataset_test\ClassA\image29.png
/dataset_test\ClassB\0__20161202_180734_0e30__-122.33973025117105_37.7672260360502.png
/dataset_test\ClassA\image24.png
/dataset_test\ClassE\pic_022.jpg
/dataset_test\ClassC\image34.png
/dataset_test\ClassB\0__20170106_180851_0e30__-122.34625465171712_37.73027249632148.png
/dataset_test\ClassC\image31.png
/dataset_test\ClassC\image6.png
/dataset_test\ClassE\pic_027.jpg
/dataset_test\ClassD\0048.jpg
/dataset_test\ClassC\image36.png
/dataset_test\ClassE\pic_038.jpg
/dataset_test\ClassC\image22.png
/dataset_test\ClassB\0__20170314_155238_0c12__-122.34813641569522_37.7602696454402.png
/dataset_test\ClassE\pic_049.jpg
/dataset_test\ClassE\pic_021.jpg
/dataset_test\ClassD\0029.jpg
/dataset_test\ClassE\pic_007.jpg
/dataset_test\ClassE\pic_016.jpg
/dataset_test\ClassA\image41.png
/dataset_test\ClassC\image27.png
/dataset_test\ClassA\image32.png
/dataset_test\ClassE\pic_014.jpg
/dataset_test\Class

RuntimeError: stack expects each tensor to be equal size, but got [600, 600, 3] at entry 0 and [20, 20, 3] at entry 1

In [ ]:
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%s' % classes[labels [j]] for j in range(4)))

In [65]:
outputs = net(images)
_,predicted = torch.max(outputs, 1)
print('Predicted: ', ' '.join('%s' % classes [predicted [j]] for j in range(4)))

NameError: name 'images' is not defined

In [66]:
correct = 0
total = 0
with torch.no_grad():
  for data in testloader:
    images, labels = data
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100* correct / total))

/dataset_test\ClassE\pic_026.jpg
/dataset_test\ClassC\image17.png
/dataset_test\ClassC\image34.png
/dataset_test\ClassC\image8.png
/dataset_test\ClassB\0__20170129_181026_0e1f__-122.40277681715277_37.830915397783336.png
/dataset_test\ClassE\pic_057.jpg
/dataset_test\ClassC\image14.png
/dataset_test\ClassA\image10.png
/dataset_test\ClassE\pic_002.jpg
/dataset_test\ClassA\image41.png
/dataset_test\ClassB\0__20170314_155238_0c12__-122.35273674958297_37.74826247058985.png
/dataset_test\ClassC\image44.png
/dataset_test\ClassA\image9.png
/dataset_test\ClassE\pic_056.jpg
/dataset_test\ClassD\0059.jpg
/dataset_test\ClassA\image16.png
/dataset_test\ClassB\0__20170414_180529_0f28__-122.34793704722254_37.76746100843631.png
/dataset_test\ClassB\0__20170414_180529_0f28__-122.35432514758647_37.75749161972382.png
/dataset_test\ClassE\pic_014.jpg
/dataset_test\ClassC\image6.png
/dataset_test\ClassA\image1.png
/dataset_test\ClassC\image23.png
/dataset_test\ClassE\pic_050.jpg
/dataset_test\ClassD\0012.j

RuntimeError: stack expects each tensor to be equal size, but got [600, 600, 3] at entry 0 and [20, 20, 3] at entry 1

In [67]:
class_correct = list(0. for i in range(10)) 
class_total = list(0. for i in range(10))

with torch.no_grad():
  for data in testloader: 
    images, labels = data
    outputs = net(images)
    _, predicted = torch.max(outputs, 1) 
    c = (predicted == labels).squeeze() 
    for i in range(4):
      label = labels[i]
      class_correct[label] += c[i].item() 
      class_total [label] += 1

for i in range(10):
  print('Accuracy of %5s: %2d %%' % (classes[i], 100* class_correct[i]/class_total[i]))

/dataset_test\ClassA\image11.png
/dataset_test\ClassD\0025.jpg
/dataset_test\ClassB\0__20161202_180734_0e30__-122.33973025117105_37.7672260360502.png
/dataset_test\ClassC\image14.png
/dataset_test\ClassB\0__20170314_155238_0c12__-122.32983736420354_37.719797138747296.png
/dataset_test\ClassC\image23.png
/dataset_test\ClassD\0009.jpg
/dataset_test\ClassE\pic_011.jpg
/dataset_test\ClassB\0__20170106_180851_0e30__-122.33658489325548_37.7609082068482.png
/dataset_test\ClassA\image47.png
/dataset_test\ClassA\image19.png
/dataset_test\ClassB\0__20170106_180852_0e30__-122.33754040846814_37.71102578191657.png
/dataset_test\ClassB\0__20160622_170157_0c64__-122.34466653421043_37.78296341833453.png
/dataset_test\ClassC\image7.png
/dataset_test\ClassE\pic_028.jpg
/dataset_test\ClassB\0__20170106_180851_0e30__-122.32845530039546_37.72960461142898.png
/dataset_test\ClassC\image47.png
/dataset_test\ClassD\0071.jpg
/dataset_test\ClassD\0028.jpg
/dataset_test\ClassD\0006.jpg
/dataset_test\ClassD\0052.j

RuntimeError: stack expects each tensor to be equal size, but got [32, 49, 4] at entry 0 and [855, 1280, 3] at entry 1